In [68]:
translation_names = ['WRJTx', 'WRJTy', 'WRJTz']
rot_names = ['WRJRx', 'WRJRy', 'WRJRz']
joint_names = [
    "R_thumb_MCP_joint1",
    "R_thumb_MCP_joint2",
    "R_thumb_PIP_joint",
    "R_thumb_DIP_joint",
    "R_index_MCP_joint",
    "R_index_DIP_joint",
    "R_middle_MCP_joint",
    "R_middle_DIP_joint",
    "R_ring_MCP_joint",
    "R_ring_DIP_joint",
    "R_pinky_MCP_joint",
    "R_pinky_DIP_joint"
]

thumb = {
    "R_thumb_MCP_joint1",
    "R_thumb_MCP_joint2",
}

In [ ]:
import numpy as np
from collections import defaultdict
import scipy.spatial.transform as transform
import numpy as np
from scipy.spatial.transform import Rotation as R

grasp_poses =np.load('/home/ubuntu/Documents/DexYCB/grasp_poses_d1_5_v0.npy', allow_pickle=True).item()
# grasp_poses =np.load('/home/ubuntu/Documents/DexYCB/grasp_poses_opt.npy', allow_pickle=True).item()
grasp_poses.keys()
# obj_idx = 3

dict_keys([210])

In [70]:
# grasp_poses[obj_idx].keys()


In [71]:
# grasp_poses[obj_idx]['target_object_name']

In [72]:
# grasp_poses[obj_idx]['robot_pose'][0]

In [73]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def euler_to_rotation_matrix(euler_angles):
    rotation = R.from_euler('XYZ', euler_angles, degrees=False)
    return rotation.as_matrix()


def quaternion_to_rotation_matrix(quaternion):
    rotation = R.from_quat(quaternion)
    return rotation.as_matrix()


def object_pose_to_matrix(position, quaternion):
    """
    Converts object pose (position and quaternion) to a 4x4 transformation matrix.
    
    Parameters:
    - position: (3,) numpy.ndarray, position [x, y, z]
    - quaternion: (4,) numpy.ndarray, [x, y, z, w] quaternion
    
    Returns:
    - transformation_matrix: (4, 4) numpy.ndarray, corresponding transformation matrix
    """
    quaternion = np.concatenate([quaternion[1:4], quaternion[0:1]]) # wxyz-> xyzw
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = position
    return transformation_matrix


def hand_pose_to_matrix(position, quaternion):
    """
    Converts object pose (position and quaternion) to a 4x4 transformation matrix.
    
    Parameters:
    - position: (3,) numpy.ndarray, position [x, y, z]
    - quaternion: (4,) numpy.ndarray, [x, y, z, w] quaternion
    
    Returns:
    - transformation_matrix: (4, 4) numpy.ndarray, corresponding transformation matrix
    """
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = position
    return transformation_matrix

In [74]:
def get_obj_centric_pose_for_opt(grasp_poses, obj_idx):
    ''' Obj pose '''
    obj_pos = grasp_poses[obj_idx]['target_pose_world'][0].p
    obj_quat = grasp_poses[obj_idx]['target_pose_world'][0].q
    object_pose = object_pose_to_matrix(obj_pos, obj_quat)
    # print(f"Object Position: {obj_pos}, Object Quaternion: {obj_quat}")
    # print(f"Object Pose: {object_pose}")

    ''' Hand pose '''
    hand_pos = grasp_poses[obj_idx]['robot_pose'][0][:3]
    hand_euler = grasp_poses[obj_idx]['robot_pose'][0][3:6]
    hand_quat = transform.Rotation.from_euler('XYZ', hand_euler, degrees=False).as_quat()
    hand_6drot = transform.Rotation.from_euler('XYZ', hand_euler, degrees=False).as_matrix()
    hand_6drot = hand_6drot[:, :2].T.ravel().tolist()
    # print(f"Hand Position: {hand_pos}, Hand Quaternion: {hand_quat}")
    # print(f"Hand 6drot: {hand_6drot}")
    # print(grasp_poses[obj_idx]['robot_pose'][0])

    ''' object-centric '''
    W_T_O = object_pose
    W_T_H = hand_pose_to_matrix(hand_pos, hand_quat)

    O_T_H = np.linalg.inv(W_T_O) @ W_T_H
    print(O_T_H)
    t_oh  = O_T_H[:3, 3]
    R_oh  = O_T_H[:3, :3]

    euler_oh = R.from_matrix(R_oh).as_euler('XYZ', degrees=False)
    print(f"Object-Centric Hand Euler (XYZ, rad): {euler_oh}")
    print(f"Object-Centric Hand Position: {t_oh}")

    hand_6drot = R_oh
    hand_6drot = hand_6drot[:, :2].T.ravel().tolist()
    hand_pos = t_oh
    hand_euler = euler_oh

    return hand_pos, hand_euler, object_pose, hand_6drot

In [75]:
grasp_poses

{210: {'target_object_name': '037_scissors',
  'target_object_idx': 17,
  'target_pose_camera': [array([ 0.29365462,  0.20150022, -0.2578915 ,  0.89814067,  0.10229845,
          -0.21143647,  0.9633028 ], dtype=float32)],
  'target_pose_world': [Pose([0.486657, 0.148532, 0.275011], [0.476852, -0.0677228, -0.686595, 0.544622])],
  'camera_pose': Pose([1.07722, 0.827201, 0.69187], [0.129591, -0.26818, -0.85521, 0.424148]),
  'robot_names': [<RobotName.inspire: 6>],
  'robot_pose': [array([ 0.56702411,  0.30217066,  0.20630386,  3.7600112 , -2.37079048,
           1.52098   ,  0.94451845,  0.97389585,  1.10423684,  1.23822463,
           1.23903787,  0.32244459,  0.99076544,  1.12944696,  1.27200862,
           1.2728739 ,  0.43014108,  0.21507054])],
  'hand_type': <HandType.right: 1>}}

In [76]:
ycb_optimize_dataset_list = []

for obj_idx in grasp_poses.keys():
    hand_pos, hand_euler, object_pose, hand_6drot = get_obj_centric_pose_for_opt(grasp_poses, obj_idx)
    robot_pose_joint = grasp_poses[obj_idx]['robot_pose'][0]
    map_idx = [0, 5, 10, 11, 1, 6, 2, 7, 3, 8, 4, 9]
    mapped_joint = [robot_pose_joint[6:][i] for i in map_idx]

    robot_joint_dict = defaultdict(float)
    for i, joint_name in enumerate(joint_names):
        robot_joint_dict[joint_name] = mapped_joint[i]
    for i, name in enumerate(translation_names):
        robot_joint_dict[name] = hand_pos[i]
    for i, name in enumerate(rot_names):
        robot_joint_dict[name] = hand_euler[i]

    ycb_optimize_dataset = defaultdict(dict)
    ycb_optimize_dataset['qpos'] = robot_joint_dict
    ycb_optimize_dataset['object_code'] = grasp_poses[obj_idx]['target_object_name']
    ycb_optimize_dataset['object_pose'] = object_pose
    ycb_optimize_dataset['hand_rot6d'] = hand_6drot
    ycb_optimize_dataset['idx'] = obj_idx

    ycb_optimize_dataset_list.append(ycb_optimize_dataset)

ycb_optimize_dataset_list

[[-0.81976489 -0.34329969  0.45840032  0.01108673]
 [ 0.19306885 -0.91922505 -0.34314681  0.08263907]
 [ 0.53917525 -0.19279689  0.81982889 -0.16683   ]
 [ 0.          0.          0.          1.        ]]
Object-Centric Hand Euler (XYZ, rad): [0.39640249 0.47619443 2.74500368]
Object-Centric Hand Position: [ 0.01108673  0.08263907 -0.16683   ]


[defaultdict(dict,
             {'qpos': defaultdict(float,
                          {'R_thumb_MCP_joint1': 0.9445184469223022,
                           'R_thumb_MCP_joint2': 0.3224445879459381,
                           'R_thumb_PIP_joint': 0.43014108031988146,
                           'R_thumb_DIP_joint': 0.21507054015994073,
                           'R_index_MCP_joint': 0.9738958477973938,
                           'R_index_DIP_joint': 0.9907654430979491,
                           'R_middle_MCP_joint': 1.1042368412017822,
                           'R_middle_DIP_joint': 1.1294469566702843,
                           'R_ring_MCP_joint': 1.2382246255874634,
                           'R_ring_DIP_joint': 1.2720086193788052,
                           'R_pinky_MCP_joint': 1.2390378713607788,
                           'R_pinky_DIP_joint': 1.272873904749155,
                           'WRJTx': 0.011086725559808506,
                           'WRJTy': 0.08263906642442853,
      

In [77]:
ycb_optimize_dataset_list

[defaultdict(dict,
             {'qpos': defaultdict(float,
                          {'R_thumb_MCP_joint1': 0.9445184469223022,
                           'R_thumb_MCP_joint2': 0.3224445879459381,
                           'R_thumb_PIP_joint': 0.43014108031988146,
                           'R_thumb_DIP_joint': 0.21507054015994073,
                           'R_index_MCP_joint': 0.9738958477973938,
                           'R_index_DIP_joint': 0.9907654430979491,
                           'R_middle_MCP_joint': 1.1042368412017822,
                           'R_middle_DIP_joint': 1.1294469566702843,
                           'R_ring_MCP_joint': 1.2382246255874634,
                           'R_ring_DIP_joint': 1.2720086193788052,
                           'R_pinky_MCP_joint': 1.2390378713607788,
                           'R_pinky_DIP_joint': 1.272873904749155,
                           'WRJTx': 0.011086725559808506,
                           'WRJTy': 0.08263906642442853,
      

In [78]:
# store dict in a specified path as npy file
import os
import json
output_path = '/home/ubuntu/Documents/DexGraspNet/data/dataset/dexycb_robot_joint_dict_210_test.npy'
if not os.path.exists(os.path.dirname(output_path)):
    os.makedirs(os.path.dirname(output_path))
np.save(output_path, ycb_optimize_dataset_list, allow_pickle=True)